In [5]:
import os
import pickle
import pathlib

from tqdm import tqdm

import numpy as np
import pandas as pd

import optuna
from optuna.samplers import CmaEsSampler, TPESampler, RandomSampler
from optuna.distributions import CategoricalDistribution

from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import make_scorer, mean_squared_error
from scipy.stats import pearsonr
from sklearn.utils import shuffle

/nobackup/users/hmbaghda/Software/miniforge3/envs/metastatic_potential/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
data_path = '/nobackup/users/hmbaghda/metastatic_potential/'
random_state = 42

n_cores = 80
os.environ["OMP_NUM_THREADS"] = str(n_cores)
os.environ["MKL_NUM_THREADS"] = str(n_cores)
os.environ["OPENBLAS_NUM_THREADS"] = str(n_cores)
os.environ["VECLIB_MAXIMUM_THREADS"] = str(n_cores)
os.environ["NUMEXPR_NUM_THREADS"] = str(n_cores)

In [7]:
def write_pickled_object(object_, file_name: str) -> None:
    if '.' in file_name:
        p = pathlib.Path(file_name)
        extensions = "".join(p.suffixes)
        file_name = str(p).replace(extensions, '.pickle')
    else:
        file_name = file_name + '.pickle'

    with open(file_name, 'wb') as handle:
        pickle.dump(object_, handle)

In [8]:
# Feature selection transformer
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, method='top_n_cv', n_features=None):
        if method not in ['top_n_cv']:#, 'all_features']:
            raise ValueError('Incorrect feature selection method implemented')
        self.method = method
        self.n_features = n_features

    def fit(self, X, y=None):
        if self.method == 'top_n_cv':
            self.coefficient_of_variation_ = np.std(X, axis=0) / np.mean(X, axis=0)
            self.top_indices_ = np.argsort(self.coefficient_of_variation_)[::-1][:self.n_features]
#         elif self.method == 'all_features':
#             self.top_indices_ = range(X.shape[1])
        return self
    def transform(self, X, y=None):
        return X[:, self.top_indices_]
    
class MeanCenterer(TransformerMixin, BaseEstimator):
    def fit(self, X, y=None):
#         self.mean_ = np.mean(X, axis=0)
        return self

    def transform(self, X, y=None):
        return X - np.mean(X, axis=0)
    
def pearson_corr_scorer(y_true, y_pred):
    return pearsonr(y_true, y_pred)[0]

class PLSRegression_X(PLSRegression):
    def transform(self, X, y=None):
        X_transformed = super().transform(X, y)
        if isinstance(X_transformed, tuple):
            X_transformed = X_transformed[0]
        return X_transformed

In [9]:
class HybridSampler(optuna.samplers.BaseSampler):
    def __init__(self, primary_sampler, fallback_sampler):
        self.primary_sampler = primary_sampler  # e.g., CmaEsSampler
        self.fallback_sampler = fallback_sampler  # e.g., TPESampler

    def infer_relative_search_space(self, study, trial):
        # Let the primary sampler define the relative search space
        return self.primary_sampler.infer_relative_search_space(study, trial)

    def sample_relative(self, study, trial, search_space):
        # Let the primary sampler handle relative sampling
        return self.primary_sampler.sample_relative(study, trial, search_space)

    def sample_independent(self, study, trial, param_name, param_distribution):
        # Use the fallback sampler for unsupported parameter types
        if isinstance(param_distribution, CategoricalDistribution):
            return self.fallback_sampler.sample_independent(study, trial, param_name, param_distribution)
        # Default to the primary sampler
        return self.primary_sampler.sample_independent(study, trial, param_name, param_distribution)

class RandomTPESampler(TPESampler):
    def __init__(self, exploration_sampler, exploration_freq=20, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.exploration_sampler = exploration_sampler
        self.exploration_freq = exploration_freq

    def sample_independent(self, study, trial, param_name, param_distribution):
        # Use the exploration_sampler periodically
        if trial.number % self.exploration_freq == 0:
            return self.exploration_sampler.sample_independent(study, trial, param_name, param_distribution)
        # Default to TPE
        return super().sample_independent(study, trial, param_name, param_distribution)


def optuna_objective(trial, X, y, inner_cv, n_cores, random_state):
    # Define feature reduction/selection method
    feature_step = trial.suggest_categorical("feature_step", ["PLS", "PCA", "FeatureSelector"])

    if feature_step == "PLS":
        steps = [
            ("mean_centering", MeanCenterer()),
            ("feature_reduction", PLSRegression_X(n_components=trial.suggest_categorical("PLS__n_components", [2, 5, 10, 25, 50, 100]))),
        ]
    elif feature_step == "PCA":
        steps = [
            ("mean_centering", MeanCenterer()),
            ("feature_reduction", PCA(n_components=trial.suggest_categorical("PCA__n_components", [2, 5, 10, 25, 50, 100]), random_state=random_state)),
        ]
    elif feature_step == "FeatureSelector":
        steps = [
            ("feature_reduction", FeatureSelector(method="top_n_cv", n_features=trial.suggest_categorical("FeatureSelector__n_features", [250, 500, 1000, 5000, 17879]))),
            ("mean_centering", MeanCenterer()),
        ]
    # Define model
    model_type = trial.suggest_categorical("model_type", ["SVR", "RFR"])
    if model_type == "SVR":
        steps.append(("model", SVR(
            kernel=trial.suggest_categorical("SVR__kernel", ["rbf", "poly"]),
            C=trial.suggest_float("SVR__C", 1e-4, 1e2, log = True),
            degree=trial.suggest_int("SVR__degree", 2, 4),
#             coef0=trial.suggest_uniform("SVR__coef0", 0, 2),
            gamma=0.001
        )))
    elif model_type == "RFR":
        steps.append(("model", RandomForestRegressor(
            n_estimators=trial.suggest_int("RFR__n_estimators", 300, 1600, step=400),
            max_features=trial.suggest_categorical("RFR__max_features", ["sqrt", "log2", 0.5, 0.75, 1]),
            max_samples=trial.suggest_categorical("RFR__max_samples", [0.25, 0.5, 0.75, None]),
            max_depth=trial.suggest_categorical("RFR__max_depth", [None, 10, 25, 50, 100, 200]),
            random_state=random_state,
            n_jobs=int(n_cores/inner_cv.n_splits)
        )))

    # Create the pipeline
    pipeline = Pipeline(steps)

    # Evaluate with cross-validation
    mse = -cross_val_score(pipeline, X, y, 
                           cv=inner_cv, 
                           scoring="neg_mean_squared_error", 
                           n_jobs=inner_cv.n_splits).mean()

#     for fold_idx, (train_idx, val_idx) in enumerate(inner_cv.split(X, y)):
#         X_train, X_val = X[train_idx], X[val_idx]
#         y_train, y_val = y[train_idx], y[val_idx]

#         # Train and evaluate the pipeline on the current fold
#         pipeline.fit(X_train, y_train)
#         y_val_pred = pipeline.predict(X_val)
#         mse = mean_squared_error(y_val, y_val_pred)

#         # Store the MSE for this fold
#         mse_scores.append(mse)

#         # Report intermediate result to Optuna
#         trial.report(np.mean(mse_scores), step=fold_idx)

#         # Check if the trial should be pruned
#         if trial.should_prune():
#             raise optuna.exceptions.TrialPruned()
    
#     return np.mean(mse_scores)

    return mse


def generate_best_pipeline(study):
    best_params = study.best_params
    steps = []
    if best_params["feature_step"] == "PLS":
        steps.append(("mean_centering", MeanCenterer()))
        steps.append(("feature_reduction", PLSRegression_X(n_components=best_params["PLS__n_components"])))
    elif best_params["feature_step"] == "PCA":
        steps.append(("mean_centering", MeanCenterer()))
        steps.append(("feature_reduction", PCA(n_components=best_params["PCA__n_components"], random_state=random_state)))
    elif best_params["feature_step"] == "FeatureSelector":
        steps.append(("feature_reduction", FeatureSelector(method="top_n_cv", n_features=best_params["FeatureSelector__n_features"])))
        steps.append(("mean_centering", MeanCenterer()))

    if "SVR__kernel" in best_params:
        steps.append(("model", SVR(
            kernel=best_params["SVR__kernel"],
            C=best_params["SVR__C"],
            degree=best_params["SVR__degree"],
#             coef0=best_params["SVR__coef0"],
            gamma=0.001
        )))
    elif "RFR__n_estimators" in best_params:
        steps.append(("model", RandomForestRegressor(
            n_estimators=best_params["RFR__n_estimators"],
            max_features=best_params["RFR__max_features"],
            max_samples=best_params["RFR__max_samples"],
            max_depth=best_params["RFR__max_depth"],
            random_state=random_state,
            n_jobs=n_cores
        )))

    best_pipeline = Pipeline(steps)
    return best_pipeline


In [10]:
X = pd.read_csv(os.path.join(data_path, 'processed',  'expr.csv'), index_col = 0).T.values
y = pd.read_csv(os.path.join(data_path, 'processed', 'metastatic_potential.csv'), index_col = 0).values.ravel()

In [11]:
outer_folds=10
inner_folds=5
n_trials = 150

In [12]:
cmaes_sampler = CmaEsSampler(seed=random_state, 
                             warn_independent_sampling=False, 
                            restart_strategy='bipop')

exploration_sampler = RandomSampler(seed=random_state)
tpe_sampler = RandomTPESampler(seed=random_state, 
                               n_startup_trials = 25,
                               exploration_sampler = exploration_sampler, 
                               exploration_freq=20 # randomly sample every n trials
                              )
# tpe_sampler = TPESampler(seed=random_state, 
#                         n_startup_trials = 20)

/nobackup/users/hmbaghda/Software/miniforge3/envs/metastatic_potential/lib/python3.13/site-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``restart_strategy`` is an experimental feature. The interface can change in the future.
  warnings.warn(


In [13]:
# outer_cv = KFold(n_splits=outer_folds, shuffle=True, random_state=random_state)
# inner_cv = KFold(n_splits=inner_folds, shuffle=True, random_state=random_state)

# results = []
# for k, (train_idx, test_idx) in enumerate(outer_cv.split(X, y)):
#     print(str(k))
#     X_train, X_test = X[train_idx], X[test_idx]
#     y_train, y_test = y[train_idx], y[test_idx]
    
    
#     pruner = optuna.pruners.SuccessiveHalvingPruner()
#     study = optuna.create_study(direction="minimize", 
#                                 sampler=HybridSampler(primary_sampler=cmaes_sampler, fallback_sampler=tpe_sampler), 
#                                pruner = pruner, 
#                                study_name = '{}_optuna'.format(k))
#     study.optimize(
#         lambda trial: optuna_objective(trial, X_train, y_train, inner_cv, n_cores, random_state),
#         n_trials=n_trials, 
#         catch=(ValueError,)
#     )
#     write_pickled_object(study, os.path.join(data_path, 'interim', study.study_name + '.pickle'))
        
#     best_pipeline = generate_best_pipeline(study)
#     best_pipeline.fit(X_train, y_train)

#     y_train_pred = best_pipeline.predict(X_train)
#     y_test_pred = best_pipeline.predict(X_test)

#     train_corr = pearsonr(y_train, y_train_pred)[0]
#     test_corr = pearsonr(y_test, y_test_pred)[0]

#     results.append({
#         "fold": k,
#         "train_corr": train_corr,
#         "test_corr": test_corr,
#         "best_params": study.best_params,
#         "inner_cv": study.trials_dataframe()
#         })
#     res_df = pd.DataFrame(results)
#     res_df.to_csv(os.path.join(data_path, 'interim', 'pipeline.csv'))

res = pd.read_csv(os.path.join(data_path, 'interim', 'pipeline.csv'), index_col = 0)
res.test_corr.mean()

np.float64(0.3542783142850661)

In [14]:
res

,fold,train_corr,test_corr,best_params,inner_cv
0,0,0.994878,0.272630,"{'feature_step': 'PLS', 'PLS__n_components': 1...",number value datetime_sta...
1,1,0.959903,0.313248,"{'feature_step': 'PLS', 'PLS__n_components': 1...",number value datetime_sta...
2,2,0.976810,0.489579,"{'feature_step': 'PLS', 'PLS__n_components': 2...",number value datetime_sta...
3,3,0.876442,0.217207,"{'feature_step': 'PLS', 'PLS__n_components': 2...",number value datetime_sta...
4,4,0.995946,0.470749,"{'feature_step': 'PLS', 'PLS__n_components': 2...",number value datetime_sta...
5,5,0.991274,0.332673,"{'feature_step': 'FeatureSelector', 'FeatureSe...",number value datetime_sta...
6,6,0.965412,0.465452,"{'feature_step': 'PLS', 'PLS__n_components': 2...",number value datetime_sta...
7,7,0.954671,0.192457,"{'feature_step': 'PCA', 'PCA__n_components': 5...",number value datetime_sta...
8,8,0.986323,0.542248,"{'feature_step': 'PLS', 'PLS__n_components': 2...",number value datetime_sta...
9,9,0.999242,0.246540,"{'feature_step': 'FeatureSelector', 'FeatureSe...",number value datetime_sta...


Select a best consensus model and re-run on new folds to see the performance:

In [15]:
for bp in res.best_params:
    print(bp)
    print('------------------------------------------------------')

{'feature_step': 'PLS', 'PLS__n_components': 10, 'model_type': 'RFR', 'RFR__n_estimators': 300, 'RFR__max_features': 'sqrt', 'RFR__max_samples': None, 'RFR__max_depth': 200}
------------------------------------------------------
{'feature_step': 'PLS', 'PLS__n_components': 10, 'model_type': 'RFR', 'RFR__n_estimators': 300, 'RFR__max_features': 0.75, 'RFR__max_samples': 0.25, 'RFR__max_depth': 10}
------------------------------------------------------
{'feature_step': 'PLS', 'PLS__n_components': 25, 'model_type': 'RFR', 'RFR__n_estimators': 300, 'RFR__max_features': 'sqrt', 'RFR__max_samples': 0.25, 'RFR__max_depth': 10}
------------------------------------------------------
{'feature_step': 'PLS', 'PLS__n_components': 2, 'model_type': 'RFR', 'RFR__n_estimators': 1500, 'RFR__max_features': 'sqrt', 'RFR__max_samples': 0.25, 'RFR__max_depth': 200}
------------------------------------------------------
{'feature_step': 'PLS', 'PLS__n_components': 25, 'model_type': 'RFR', 'RFR__n_estimators

In [16]:
best_steps = [
    ("mean_centering", MeanCenterer()),
    ("feature_reduction", PLSRegression_X(n_components=10, scale = True)),
]
best_steps.append(("model", RandomForestRegressor(
    n_estimators=300,
    max_features='sqrt',
    max_samples=0.25,
    max_depth=10, # tie between 200 and 10
    random_state=random_state,
    n_jobs=n_cores
)))
best_pipeline = Pipeline(best_steps)

In [17]:
X = pd.read_csv(os.path.join(data_path, 'processed',  'expr.csv'), index_col = 0).T.values
y = pd.read_csv(os.path.join(data_path, 'processed', 'metastatic_potential.csv'), index_col = 0).values.ravel()

In [18]:
outer_cv = KFold(n_splits=10, shuffle=True, random_state=random_state+1)


res = {}

results = []
for k, (train_idx, test_idx) in tqdm(enumerate(outer_cv.split(X, y))):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    best_pipeline.fit(X_train, y_train)

    y_train_pred = best_pipeline.predict(X_train)
    y_test_pred = best_pipeline.predict(X_test)

    train_corr = pearsonr(y_train, y_train_pred)[0]
    test_corr = pearsonr(y_test, y_test_pred)[0]

    results.append({
        "fold": k,
        "train_corr": train_corr,
        "test_corr": test_corr,
        })
    best_res_df = pd.DataFrame(results)
    res[k] = {'test': y_test, 'pred': y_test_pred, 'train': y_train}

10it [00:12,  1.30s/it]


In [23]:
best_res_df

,fold,train_corr,test_corr
0,0,0.973936,0.322485
1,1,0.979850,0.520824
2,2,0.973400,0.268299
3,3,0.975479,0.570978
4,4,0.970206,-0.012856
5,5,0.978579,0.379738
6,6,0.973479,0.372561
7,7,0.971110,0.713619
8,8,0.971442,0.513119
9,9,0.976556,0.291682


In [20]:
best_res_df.test_corr.mean()

np.float64(0.3940447207902385)